In [ ]:
##README
"""
file used to train a model for eye-tracking
"""

In [17]:
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.models

import tqdm
import matplotlib.pyplot as plt
import csv
import os
import cv2


In [18]:
!pip install torch-summary

from torchsummary import summary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Params

In [12]:
root = ""
train_csv = "/content/sample_data/california_housing_test.csv"
val_csv = "/content/sample_data/california_housing_test.csv"

In [6]:
PRMS = {
    'N_train_entries' : 1000,
    'batch_size' : 4,
    'N_validation_entries' : 100,

    'img_size' : 224,
}

In [30]:
class Eye_Dataset(Dataset):
    def __init__(self, path,transform=None):
        self.path = path
        self.transform = transform #maybe for resizing and colordistortion for augmentation
        
        self.file = open(path)
        self.header = self.file.readline()
        self.Y = np.array([torch.tensor([float(k) for k in x.split(',')][:-1]) for x in self.file.readlines()[1:]])
        self.file.close()
        
        self.file = open(path)
        self.imgpath = np.array([torch.tensor(int(x.split(',')[-1])-1) for x in self.file.readlines()[1:]])
        self.file.close()
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        image = cv2.imread(self.imgpath[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform:
            image = self.transform(image=image)
        
        return (image,self.Y[idx])

In [31]:
traindataset = Eye_Dataset(train_csv)
valdataset = Eye_Dataset(val_csv)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


ValueError: ignored

In [29]:
trainloader = DataLoader(traindataset, batch_size= PRMS['batch_size'], shuffle=False,num_workers=4)
valloader = DataLoader(valdataset, batch_size= PRMS['batch_size'], shuffle=False,num_workers=4)

NameError: ignored

# Model

In [25]:
beautiful_eyes_model = torchvision.models.resnet18()
beautiful_eyes_model.fc = nn.Linear(512,2, bias=True) #changing last layer to x and y channels

In [28]:
lossfn = nn.MSELoss(size_average=None, reduce=None, reduction='sum')

# Training algorithms

In [ ]:
def trainer(model,loss_fn=lossfn,epoch=5,rate=1e-3):

    optimizer = torch.optim.Adam(params = model.parameters(), lr = rate)
    # localBatch = torch.utils.data.DataLoader(dataset, batch_size=batch_size,shuffle=True)

    for i in tqdm.notebook.tnrange(epoch, desc = 'epoch'):

        for batch,(x,y) in enumerate(trainloader):
            y_pred = model.forward(x)
            loss = loss_fn(y_pred,y)
            optimizer.zero_grad()
            loss.backward()

            optimizer.step()

In [ ]:
def avg_distance_error(model):
    distance = 0
  # dataloader = torch.utils.data.DataLoader(dataset_test, batch_size=1)
    for x, y in tqdm.tqdm(valloader, desc='DataLoader', leave=False):
        ypred = model.forward(x)[0]
        distance += torch.norm(ypred-y)
    return distance/PRMS['N_validation_entries']

# Training

In [ ]:
trainer(model = beautiful_eyes_model, loss_fn = lossfn, epoch = 5, rate = 1e-3)

In [ ]:
avg_distance_error(beautiful_eyes_model)